In [6]:
import json
import numpy as np


raw_data = {
   "drawing":{
      "strokes":[
         [
            {
               "x":100,
               "y":200,
               "a":15,
               "p":0.4
            },
            {
               "x":300,
               "y":400,
               "a":90,
               "p":0.5
            },
            {
               "x":400,
               "y":200,
               "a":125,
               "p":1.
            }
         ],
         [
            {
               "x":600,
               "y":400,
               "a":45,
               "p":0.2
            },
            {
               "x":600,
               "y":300,
               "a":45,
               "p":0.3
            }
         ]
      ]
   }
}

class Drawing_processor(object):
   """
   Python class to convert a dictionary and into a JSON format and back
   """
   def __init__(self,pressure_factor= -1, base_z = 15, safe_z_val = 5 ):
      """
      Args:
           pressure_factor (float): a quoefficient to make thickinsses based on pressure on paper
           base_z (float): the z value of the paper that the robot draws on
           safe_z_val (float):  value that is going to be added to the base_z while the 
                                 drawing tool is not supposed to write on the paper
      """
      self.default_path = "../data/path_data.json"
      
      self.pressure_factor = pressure_factor
      self.safe_z_val = safe_z_val
      self.base_z = base_z

      self.json_object = None

   def dic_to_json(self, dic_data):
      """
      Converts a dictionary to json format.

      Args: 
         dic_data (dict): data to be converted to JSON

      returns:
         self.json_object (string): dic_data as a string in the json structure
      """
      self.json_string = json.dumps(dic_data, indent = 4) 
      return self.json_object

   
   def json_to_dict(self, json_data= None, json_path=None):
      """
      Converts a json fomat data to a dictionary.
      If given the json_data, it returns the dictionary of it
      If given the json_path, it loads the file, then returns the dictionary
      
      Args: 
         json_data: a serialized json object
         json_path (string): a path that holds the json data as a file, i.e.: "../data/path_data.json"
      
      retunrs:
         dictionary_data (dict): a dictionary holding the data from the JSON file or data
      """

      if json_data == None:
         # in case of no path and no data, 
         # it loads a file from the default path 
         if json_path == None:
            json_path = self.default_path

         raw_data = open(json_path,)
         dictionary_data = json.load(raw_data)
      
      # if given json_data, it always return the dic
      else:
         dictionary_data = json.load(json_data)

      return dictionary_data
      

   def save_json(self, json_data, path=None):
      """
      Writes the json_data to a file
      
      Args: 
         json_data: a serialized json object
         path (string): a path to write the file on, i.e.: "../data/path_data.json"
               if no path given, it writes it on the default path 
      
      returns:
         None
      """

      if path == None:
         path = self.default_path

      with open(path, 'w') as outfile:
            json.dump(json_data, outfile)

   def extract_ploylines(self, json_path):
      """
      Extract all the polylines in a json drawing 
      
      Args: 
         json_path (string): a path to load the file from, i.e.: "../data/path_data.json" 
      
      returns:
         polyLines (nparray): an array of all polylines
      """

      drawing_data = self.json_to_dict(json_path= json_path)

      strokes = drawing_data['drawing']['strokes']

      polyLines = [pl for pl in strokes]
      polyLines = [self.get_targest_from_polyline(polyLine) for polyLine in polyLines]
      polyLines = np.asarray(polyLines, dtype=object)
      
      return polyLines
         

   def get_targest_from_polyline(self, polyLine):
      """
      Extract all the targest from a given polyline

      Args:
         polyLine (list of dictionaries): the polyline data, i.e.: [{"x": 600, "y": 400, "a": 45, "p": 0.2}, 
                                                    {"x": 600, "y": 300, "a": 45, "p": 0.3}]
      returns:
         point_in_polyline (numpy.ndarray): all the points in the polyline i.e.:
                                                   [[600. 400. 45. 0.2]
                                                    [600. 300. 45. 0.3]]

      """
      point_in_polyline = [np.asarray(list(point.values())) for point in polyLine]
      point_in_polyline = np.asarray(point_in_polyline)
      return point_in_polyline

   def draw(self, arm, drawing):
      """
      Send drawing commands to the robot arm
      Args:
         arm (Arm): arm to run the drawing on 
         drawing (numpy.ndarray): list of polylines to draw, i.e.:
                                    [array([[100. , 200. ,  15. ,   0.4],
                                            [300. , 400. ,  90. ,   0.5],
                                            [400. , 200. , 125. ,   1. ]])
                                    array([[6.0e+02, 4.0e+02, 4.5e+01, 2.0e-01],
                                           [6.0e+02, 3.0e+02, 4.5e+01, 3.0e-01]])]
      returns:
         None
      """
      # design the process to go up and down between the targest 
      arm.set_module_type(6)

      # loop over polylines
      for i, poly_line in enumerate(drawing):

         # at any position, first make sure the pen is away
         # from the paper
         current_pose = arm.get_current_position()
         arm.move_to(current_pose[0], current_pose[1], self.safe_z_val)

         for j, target in enumerate(poly_line):
            x, y, a, p = target
            #print ("Target values: x:{}, y:{}, a:{}, p:{}".format( x, y, a, p))
            
            if j ==0:
               # moves towards the first target, without changin z value
               # to avoid touching paper
               arm.move_to(x, y, self.safe_z_val)

            # always rotate to the correct azimuth before touching paper
            arm.rotate_to(a)
            # move to the target
            arm.move_to(x, y, self.base_z + p*self.pressure_factor)
         
         # moves up from paper after the last target
         arm.move_to(x, y, self.safe_z_val)
         print ("<<< polyline {} finished >>> \n".format(i))

class Robot_arm(object):
   """
   A dummy class to work with this file
   """
   def __init__(self):
      pass
   
   def set_module_type(self,module_type):
      if module_type == 6:
         print ("Module set to rotary module")
      else:
         print ("Module is not set to rotary!")
      print ("-----------------------------------------------------")
      print ("\n")
   def move_to(self,  x_loc, y_loc, p_val):
      print (("M \t{}\t{}\t{}").format(x_loc, y_loc, p_val))

   def rotate_to(self, r):
      print ("R \t{}".format(r))

   def get_current_position(self):
      x, y, z, e, a, b, c = [0,1,2,3,4,5,6]
      return x, y, z, e, a, b, c

In [7]:
arm = Robot_arm()
dp = Drawing_processor()
dp.save_json(json_data= raw_data,path= "../data/path_data_test.json")
polyLines = dp.extract_ploylines("../data/path_data_test.json")
dp.draw(arm, polyLines)

Module set to rotary module
-----------------------------------------------------


M 	0	1	5
M 	100.0	200.0	5
R 	15.0
M 	100.0	200.0	14.6
R 	90.0
M 	300.0	400.0	14.5
R 	125.0
M 	400.0	200.0	14.0
M 	400.0	200.0	5
<<< polyline 0 finished >>> 

M 	0	1	5
M 	600.0	400.0	5
R 	45.0
M 	600.0	400.0	14.8
R 	45.0
M 	600.0	300.0	14.7
M 	600.0	300.0	5
<<< polyline 1 finished >>> 



[array([[100. , 200. ,   0. ,   0.4],
        [300. , 400. ,   0. ,   0.5],
        [400. , 200. ,   0. ,   0.6]])
 array([[6.0e+02, 4.0e+02, 4.5e+01, 2.0e-01],
        [6.0e+02, 3.0e+02, 4.5e+01, 3.0e-01]])]
